In [8]:
class Online_SGD:
    import warnings
    warnings.filterwarnings("ignore")
    import numpy as np
    import pandas as pd
    
    def __init__(self,learning_rate=0.2,n_epochs=100, k=40,damp_factor=1.02,tol=1e-2,w_file="w.txt",b_file="b.txt"):
        self.learning_rate = learning_rate
        self.n_epochs=n_epochs
        self.k=k
        self.damp_factor=damp_factor
        self.tol=tol
        self.w_file=w_file
        self.b_file=b_file
        
    def fit_online(self,X,Y):
        
        self.w = np.loadtxt(self.w_file).reshape(1, X.shape[1])  # Reading weights and bias
        self.b = np.loadtxt(self.b_file).reshape(1, 1)
        
        for i in range(X.shape[0]):
            x=(np.array(X))[i].reshape(1,X.shape[1])
            y=(np.array(Y))[i].reshape(1,1)
            Lw = np.dot((y - np.dot(x, self.w.T) - self.b),x)
            Lb = (y - np.dot(x, self.w.T) - self.b)
            self.w = self.w + self.learning_rate * Lw
            self.b = self.b + self.learning_rate * Lb
            self.learning_rate = self.learning_rate / self.damp_factor
        
        a_file = open(self.w_file, "w")
        for row in self.w:
            np.savetxt(a_file, row)
        a_file.close()
        b_file = open(self.b_file, "w")
        for row in self.b:
            np.savetxt(b_file, row)
        b_file.close()
 
        return self.w, self.b
    
    def fit_regression(self,X,Y):
        from sklearn.linear_model import LinearRegression
        reg = LinearRegression()
        reg.fit(X, Y)
        self.w=reg.coef_.reshape(1,X.shape[1])
        self.b=reg.intercept_.reshape(1,1)
        a_file = open(self.w_file, "w")
        for row in self.w:
            np.savetxt(a_file, row)
        a_file.close()
        b_file = open(self.b_file, "w")
        for row in self.b:
            np.savetxt(b_file, row)
        b_file.close()
        
        return self.w,self.b
    
    def predict(self,X):
        self.w = np.loadtxt(self.w_file).reshape(1, X.shape[1])  # Reading weights and bias
        self.b = np.loadtxt(self.b_file).reshape(1, 1)
        m=np.dot(X, self.w.T) + self.b
        n=m.reshape(-1,)
        
        return n
        
    
#     def fit_batch(self,X,Y):
#         self.w = np.zeros((1, X.shape[1]))  # Randomly initializing weights
#         self.b = np.zeros((1, 1))
#         n=1
#         while n<=self.n_epochs:
#             temp = pd.concat([pd.DataFrame(X), pd.DataFrame(Y)], axis=1)
#             temp2=temp.sample(self.k)
#             X_tr = temp2.iloc[:,0:-1]
#             Y_tr = temp2.iloc[:,-1]
# #             print("shape of x_tr",X_tr.shape)
# #             print("temp shape",temp.shape,temp2.shape)
#             Lw=np.zeros((1, X_tr.shape[1]))
#             Lb=np.zeros((1, 1))
            
#             for i in range(self.k):
#                 x=(np.array(X_tr))[i].reshape(1,X_tr.shape[1])
#                 y=(np.array(Y_tr))[i].reshape(1,1)
# #                 print("shape of x",x.shape)
# #                 print("shape of y",y.shape)
#                 Lw = Lw+(np.dot((y - np.dot(x, self.w.T) - self.b),x))
#                 Lb = Lb+((y - np.dot(x, self.w.T) - self.b))
#             self.w = self.w + self.learning_rate * Lw
#             self.b = self.b + self.learning_rate * Lb
#         #             print("W",self.w)
#         #             print("b",self.b)
            
            
#             self.learning_rate = self.learning_rate / self.damp_factor
#             n=n+1
#         return self.w,self.b
                     
    


In [9]:
from math import sqrt
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [10]:
dfX = pd.read_csv('X_VAR3.csv')
dfY = pd.read_csv('Y_VAR3.csv')
Data_X=dfX
Data_Y=dfY.iloc[:,0]
Data_X_train, Data_X_test, Data_y_train, Data_y_test = train_test_split(Data_X, Data_Y, test_size=0.3, random_state=0)

In [11]:
lr=Online_SGD(learning_rate=0.2,damp_factor=1.02)
w_reg,b_reg=lr.fit_regression(Data_X_train,Data_y_train)
Data_y_pred2 = lr.predict(Data_X_test)

In [12]:
W=np.array([[0.5,0.7,-0.1]])
B=np.array([[0.0001]])

In [13]:
Dict2={"Predicted":list(Data_y_pred2),"Actual":list(Data_y_test)}
result2=pd.DataFrame(Dict2)
result2

,Predicted,Actual
0,1.966641,2.380171
1,2.537527,3.314087
2,-1.703218,-0.855238
3,-1.467727,-1.800950
4,0.023120,-0.509179
...,...,...
5251,-3.219460,-3.378119
5252,2.180031,0.576812
5253,-4.387519,-4.076459
5254,0.867203,0.596689


In [17]:
def coeff_error(w):
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    return(sqrt(mean_squared_error(W,w)))
def bias_error(b):
    return(sqrt(mean_squared_error(B,b)))
def pred_error(x,y):
    return np.sum(np.abs(x-y) / y.shape[0])

In [18]:
coeff_error(w_reg)

0.004587824239189246

In [19]:
pred_error(Data_y_test,Data_y_pred2)

0.7986794128951884

In [ ]:
grid={"learning_rate":[],"damp_factor":[],"error":[]}
for i in range(5,100,5):
    for j in range(80,130,5):
        lr=Online_SGD(learning_rate=i/100,damp_factor=j/100)
        lr.fit(Data_X_train,Data_y_train)
        grid['learning_rate'].append(i/100)
        grid['damp_factor'].append(j/100)
        z = lr.predict(Data_X_test)
        error=np.sum(np.abs(Data_y_test - z) / Data_y_test.shape[0])
        grid['error'].append(error)
grid_df=pd.DataFrame(grid)
grid_df

# Visualising Prediction Error(taking 5 samples and Online Learning)

In [ ]:
rslt_df = grid_df.sort_values(by = 'error',ascending = False)
rslt_df

In [15]:
x1=Data_X_test.iloc[0:5,:]
y1=Data_y_test.iloc[0:5,:]
x2=Data_X_test.iloc[5:10,:]
y2=Data_y_test.iloc[5:10,:]

In [27]:
lr=Online_SGD(learning_rate=0.005,damp_factor=1.02)
w1,b1=lr.fit_regression(Data_X_train,Data_y_train)
w2,b2=lr.fit_online(s,t)

In [28]:
w1,w2

(array([[ 0.50264849,  0.69275604, -0.0980882 ]]),
 array([[ 0.51560364,  0.6981122 , -0.08304536]]))

In [24]:
b1,b2

(array([[-0.00194493]]), array([[0.06797855]]))